# ACS6420 Advanced project
## Rolls-Royce Undergraduate project - Anomaly detection in discrete sequences
### Author: Aldair M Silva
### Supervisor: Visakan
Date created: 13/10/2021

In [2]:
from datetime import date
print('Updated: ', date.today().strftime('%d/%m/%Y'))

Updated:  11/02/2022


Import libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import urllib.parse
from sklearn import tree
from sklearn import metrics
import io
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix

Import data

In [4]:
normal_raw = 'normalTrafficTraining.txt'
anomaly_raw = 'anomalousTrafficTest.txt'

normal_parse = 'normalRequestTraining.txt'
anomaly_parse = 'anomalousRequestTest.txt'

## Prototype 1

File handling functions

In [6]:
# File parsing function
def parse_file(file_in, file_out):
    fin = open(file_in)
    fout = io.open(file_out, "w", encoding="utf-8")
    lines = fin.readlines()
    res = []
    
    for i in range(len(lines)):
        line = lines[i].strip()
        if line.startswith("GET"):
            res.append("GET" + line.split(" ")[1])
        elif line.startswith("POST") or line.startswith("PUT"):
            url = line.split(' ')[0] + line.split(' ')[1]
            j = 1
            while True:
                if lines[i + j].startswith("Content-Length"):
                    break
                j += 1
            j += 1
            data = lines[i + j + 1].strip()
            url += '?' + data
            res.append(url)
    for line in res:
        line = urllib.parse.unquote(line).replace('\n','').lower()
        fout.writelines(line + '\n')
    print ("finished parse ",len(res)," requests")
    fout.close()
    fin.close()

# File reading function
def loadData(file):
    with open(file, 'r', encoding="utf8") as f:
        data = f.readlines()
    result = []
    for d in data:
        d = d.strip()
        if (len(d) > 0):
            result.append(d)
    return result


In [15]:
normal_data = parse_file(normal_raw,normal_parse)
anomaly_data = parse_file(anomaly_raw,anomaly_parse)

finished parse  36000  requests
finished parse  25065  requests


In [16]:
from sklearn.utils import shuffle

In [53]:
good_requests = loadData('normalRequestTraining.txt')
bad_requests = loadData('anomalousRequestTest.txt')

all_requests = bad_requests + good_requests
yBad = [1] * len(bad_requests)
yGood = [0] * len(good_requests)
y = yBad + yGood

Doc2Vector

In [54]:
vectorizer = TfidfVectorizer(min_df=0.0, analyzer="char", sublinear_tf=True, ngram_range=(3, 3))
X = vectorizer.fit_transform(all_requests)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
pd.DataFrame(X_train[:])
X.shape

(61065, 28847)

In [79]:
X_dense = X_train[:5,:20].todense()
np.unique(X_dense[1,:])


matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])

In [78]:
y_train[1]

0

In [36]:
X_dense = X.todense()
X_dense.shape

(61065, 28847)

In [46]:
#X_dense[0,:].unique()

import csv

# Example.csv gets created in the current working directory
with open('Example.csv', 'w', newline = '') as csvfile:
	my_writer = csv.writer(csvfile, delimiter = ' ')
	my_writer.writerow(X)


KeyboardInterrupt: 

Logistic regression model

In [55]:
lgs = LogisticRegression()
lgs.fit(X_train, y_train)
y_pred = lgs.predict(X_test)
score_test = metrics.accuracy_score(y_test, y_pred)
matrix = confusion_matrix(y_test, y_pred)

print ("Score Logistic Regression :",score_test)
print ("Confusion Matrix: ")
print (matrix)

Score Logistic Regression : 0.9808416571147863
Confusion Matrix: 
[[3519   41]
 [  76 2471]]


C:\Users\Audaz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Decision tree model

In [17]:
dtc = tree.DecisionTreeClassifier()
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
score_test = metrics.accuracy_score(y_test, y_pred)
matrix = confusion_matrix(y_test, y_pred)
print ("Score Decesion Tree :",score_test)
print ("Confusion Matrix: ")
print (matrix)


Score Decesion Tree : 0.5469133780907156
Confusion Matrix: 
[[2882  728]
 [2039  458]]


SVM model

In [ ]:
linear_svm=LinearSVC(C=1)
linear_svm.fit(X_train, y_train)
y_pred = linear_svm.predict(X_test)
score_test = metrics.accuracy_score(y_test, y_pred)
matrix = confusion_matrix(y_test, y_pred)
print ("Score Linear SVM :",score_test)
print ("Confusion Matrix: ")
print (matrix)


Random forest model

In [142]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
score_test = metrics.accuracy_score(y_test, y_pred)
print ("Score Random Forest :",score_test)
print ("Confusion Matrix: ")
print (matrix)

## Prototype 2

Import libraries

In [ ]:
from sklearn.feature_selection import SelectFromModel